In [1]:
import os

import SimpleITK as sitk
import pydicom as pydicom

import pkutils as pkutils

import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests
from scipy.optimize import curve_fit, minimize
from sklearn.metrics import r2_score

import pandas as pd

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from tqdm import tqdm

import math
from concurrent.futures import ThreadPoolExecutor

NUM_WORKERS = 16

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
DCE_DIR = "/home/thulasiseetha/research/dataset/curated/LungMR/dce"

In [4]:
time_arr = np.load("timeArray.npy", allow_pickle=True).item()

stat_df = pd.read_csv("stat_df.csv")

stat_df.head()

,pid,ref_id,age,sex,weight,height,contrast_agent,slice_thickness,voxel_spacing,img_matrix,...,TE,FA,FOV,num_of_phases,acquisition_period,manufacturer,version,field_strength,pdl1>=1%,pdl1>=50%
0,2021064,paz40,067Y,F,49,1.69,DOTAREM,3,"[3.12, 3.12, 3.0]","[128, 128, 32]",...,0.74,25,"[400.0, 400.0, 96.0]",30,1.47,SIEMENS,syngo MR E11,1.5,1,1
1,2565228,paz9,060Y,M,88,1.80,DOTAREM,3,"[3.12, 3.12, 3.0]","[128, 128, 36]",...,0.73,25,"[400.0, 400.0, 108.0]",30,1.54,SIEMENS,syngo MR E11,1.5,0,0
2,2780061,paz3,056Y,M,84,1.75,DOTAREM,3,"[3.12, 3.12, 3.0]","[128, 128, 32]",...,0.74,25,"[400.0, 400.0, 96.0]",30,1.47,SIEMENS,syngo MR E11,1.5,0,0
3,2737310,paz35,070Y,M,82,1.75,DOTAREM,3,"[3.12, 3.12, 3.0]","[128, 128, 36]",...,0.73,25,"[400.0, 400.0, 108.0]",30,1.54,SIEMENS,syngo MR E11,1.5,1,0
4,2776694,paz24,067Y,M,65,1.00,DOTAREM,3,"[3.0, 3.0, 3.0]","[128, 128, 32]",...,0.74,25,"[384.0, 384.0, 96.0]",30,1.47,SIEMENS,syngo MR E11,1.5,0,0


In [5]:
sample_ids = stat_df.ref_id.to_list()

### PK Modelling on Mean Tumor Signal

##### Estimating Parameters

In [6]:
verbose = False

pk_df = {"sample_id":[], "r2":[], "t0":[], "ktrans":[], "kep":[], "ve":[], "vp":[], "iauc30":[], "iauc30_bn":[], "iauc60":[], "iauc60_bn":[], "iauc90":[], "iauc90_bn":[], "pdl1>=1%":[], "pdl1>=50%":[]}

for sample_id in tqdm(sample_ids, position=0):
    
    row_dict = stat_df[stat_df.ref_id==sample_id].iloc[0].to_dict()

    T1_0 = 1200 * 1e-3  #original in milliseconds https://pubmed.ncbi.nlm.nih.gov/28027756 to seconds

    TR = row_dict["TR"] * 1e-3  # in milliseconds to seconds #not really patient specific, from an arbitrary patient

    FA_deg = row_dict["FA"] #this is not exactly patient specific, value from an arbitrary patient

    r1 = 3.6 #L/mmol/sec

    t_arr = time_arr[sample_id]

    nt = len(t_arr)

    sitk_img = sitk.ReadImage(os.path.join(DCE_DIR, "images", sample_id, "mc_img4D.nii.gz"))
    sitk_tumor_mask = sitk.JoinSeries([sitk.ReadImage(os.path.join(DCE_DIR, "masks", sample_id, "mask.nii.gz"))]*nt)

    img_arr = sitk.GetArrayFromImage(sitk_img)
    tumor_mask_arr = sitk.GetArrayFromImage(sitk_tumor_mask)

    tumor_img_arr = np.ma.array(img_arr, mask=(tumor_mask_arr==0))
    tumor_T1w_t = np.ma.mean(tumor_img_arr, axis=(1,2,3)).data# np.ma.masked_array(img_arr, tumor_mask_arr==0).mean(axis=(1,2,3)).data
    Ct_t = pkutils.T1w_to_Con(tumor_T1w_t, T1_0, TR, FA_deg, r1, 1)
    
   
    aif_fn = pkutils.georgiou_aif        
    bat_index = pkutils.PeakGradient_BATModel().run(t_arr, tumor_T1w_t, pkutils.search_interval(tumor_T1w_t)).bat_index


    # extended
    t0_0 = t_arr[bat_index]
    vp_0 = 0.0025 #fractional volume
    ktrans_0 = 0.058/60 #in seconds
    ve_0 = 0.2 #fractional volume

    p_init = [t0_0, vp_0, ktrans_0, ve_0]
    p_lower = (-300.0, 1e-4, 60e-12/60, 0.01)
    p_upper = (300.0, 1.0, 3/60, 1.0)


    p_opt, p_cov = curve_fit(pkutils.delayCorrectedExtendedTofts(aif_fn), t_arr, Ct_t, p0=p_init, bounds = [p_lower, p_upper], maxfev=100000)#, bounds=bounds)
    extended_t0_opt, extended_vp_opt, extended_ktrans_opt, extended_ve_opt = p_opt
    extended_Ct_t = pkutils.delayCorrectedExtendedTofts(aif_fn)(t_arr, *p_opt)
    extended_r2 = r2_score(Ct_t, extended_Ct_t)
    
    
    r2 = extended_r2
    t0 = extended_t0_opt
    vp = extended_vp_opt
    ktrans = extended_ktrans_opt
    ve = extended_ve_opt
    kep = ktrans/ve
    iaucx_dict = {x:pkutils.get_iAUCx_with_functionalAIF(pkutils.ExtendedTofts, aif_fn, p_opt[1:], t_arr, t0, x=x) for x in [30,60,90]}
    iaucx_bn_dict = {x:pkutils.get_iAUCx_bn_with_functionalAIF(pkutils.ExtendedTofts, aif_fn, p_opt[1:], t_arr, t0, x=x) for x in [30,60,90]}
    
    pk_df["sample_id"].append(sample_id)
    pk_df["r2"].append(r2)
    pk_df["t0"].append(t0)
    pk_df["ktrans"].append(ktrans*60)
    pk_df["kep"].append(kep*60)
    pk_df["ve"].append(ve)
    pk_df["vp"].append(vp)
    
    for x in [30,60,90]:
        pk_df[f"iauc{x}"].append(iaucx_dict[x])
        pk_df[f"iauc{x}_bn"].append(iaucx_bn_dict[x])
        
    pk_df["pdl1>=1%"].append(int(row_dict["pdl1>=1%"]))
    pk_df["pdl1>=50%"].append(int(row_dict["pdl1>=50%"]))
    
    if verbose:
        plt.title(sample_id)
        plt.plot(t_arr, Ct_t, 'k.-', label="Measured")
        plt.plot(t_arr, extended_Ct_t, ".--", label=f"Extended Tofts Fit, r2={extended_r2:0.2}, ktrans={extended_ktrans_opt*60:0.2}")
        plt.legend(loc='upper left')
        plt.show()

pk_df = pd.DataFrame(pk_df)

100%|███████████████████████████████████████████████████████████████| 38/38 [00:16<00:00,  2.26it/s]


##### Mann-Whitney U-test (PDL1>=1%)

In [7]:
pvalue_df = {"feat":[], "p_value":[]}

label_column = "pdl1>=1%"

for feat in [ "ktrans", "ve", "kep", "vp",  "iauc60"]:

    x1 = pk_df[(pk_df[label_column]==0)][feat].to_list()
    x2 = pk_df[(pk_df[label_column]==1)][feat].to_list()

    u, p_value = stats.mannwhitneyu(x1, x2)

    roc_auc = u/(len(x1)*len(x2))

    pvalue_df["feat"].append(feat)
    pvalue_df["p_value"].append(p_value)


    # print(bat_method, aif_fn, feat, p_value, roc_auc)

pvalue_df = pd.DataFrame(pvalue_df)
_, pvalue_df["corr_p_value"], _, _ = multipletests(pvalue_df.p_value, method="fdr_bh")
display(pvalue_df)

,feat,p_value,corr_p_value
0,ktrans,0.805563,0.926454
1,ve,0.735015,0.926454
2,kep,0.926454,0.926454
3,vp,0.267995,0.926454
4,iauc60,0.600921,0.926454


##### Mann-Whitney U-test (PDL1>=50%)

In [13]:
pvalue_df = {"feat":[], "p_value":[]}

label_column = "pdl1>=50%"

for feat in ["ktrans", "ve", "kep", "vp", "iauc60"]:

    x1 = pk_df[(pk_df[label_column]==0)][feat].to_list()
    x2 = pk_df[(pk_df[label_column]==1)][feat].to_list()

    u, p_value = stats.mannwhitneyu(x1, x2)

    roc_auc = u/(len(x1)*len(x2))

    pvalue_df["feat"].append(feat)
    pvalue_df["p_value"].append(p_value)

    # print(bat_method, aif_fn, feat, p_value, roc_auc)

pvalue_df = pd.DataFrame(pvalue_df)
_, pvalue_df["corr_p_value"], _, _ = multipletests(pvalue_df.p_value, method="fdr_bh")
display(pvalue_df)

,feat,p_value,corr_p_value
0,ktrans,0.016395,0.048408
1,ve,0.441756,0.552195
2,kep,0.019363,0.048408
3,vp,0.666636,0.666636
4,iauc60,0.048927,0.081544


### PK Modelling on Voxel-wise Tumor Signal

In [14]:
OUT_DIR = "pk_maps"

In [15]:
def align_header(ref_img, input_img):
    
    input_img.SetOrigin(ref_img.GetOrigin())
    input_img.SetSpacing(ref_img.GetSpacing())
    input_img.SetDirection(ref_img.GetDirection())
    
    return input_img

In [16]:
def generate_pk_map(sample_id):
    
    global pbar, stat_df

    out_dir = os.path.join(OUT_DIR, sample_id)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    row_dict = stat_df[stat_df.ref_id==sample_id].iloc[0].to_dict()

    aif_fn = pkutils.georgiou_aif

    T1_0 = 1200 * 1e-3  #original in milliseconds https://pubmed.ncbi.nlm.nih.gov/28027756 to seconds

    TR = row_dict["TR"] * 1e-3  # in milliseconds to seconds #not really patient specific, from an arbitrary patient

    FA_deg = row_dict["FA"] #this is not exactly patient specific, value from an arbitrary patient

    r1 = 3.6 #L/mmol/sec

    t_arr = time_arr[sample_id]

    nt = len(t_arr)

    sitk_img = sitk.ReadImage(os.path.join(DCE_DIR, "images", sample_id, "mc_img4D.nii.gz"))
    sitk_tumor_mask = sitk.ReadImage(os.path.join(DCE_DIR, "masks", sample_id, "mask.nii.gz"))

    img_arr = sitk.GetArrayFromImage(sitk_img)
    tumor_mask_arr = sitk.GetArrayFromImage(sitk_tumor_mask)

    tumor_img_arr = np.ma.array(img_arr, mask=(np.stack([tumor_mask_arr]*nt)==0))
    tumor_T1w_t = np.ma.mean(tumor_img_arr, axis=(1,2,3)).data# np.ma.masked_array(img_arr, tumor_mask_arr==0).mean(axis=(1,2,3)).data

    bat_index = pkutils.PeakGradient_BATModel().run(t_arr, tumor_T1w_t, pkutils.search_interval(tumor_T1w_t)).bat_index #This way of initialization gives better fit, no convergence error

    img_arr = sitk.GetArrayFromImage(sitk_img) #TxDxWxH
    tumor_mask_arr = sitk.GetArrayFromImage(sitk_tumor_mask)

    D, W, H = tumor_mask_arr.shape

    map_dict = {}
    map_dict["vp"] = np.zeros_like(tumor_mask_arr, dtype=np.float32)
    map_dict["ktrans"] = np.zeros_like(tumor_mask_arr, dtype=np.float32)
    map_dict["ve"] = np.zeros_like(tumor_mask_arr, dtype=np.float32)
    map_dict["kep"] = np.zeros_like(tumor_mask_arr, dtype=np.float32)
    map_dict["t0"] = np.zeros_like(tumor_mask_arr, dtype=np.float32)
    map_dict["r2"] = np.zeros_like(tumor_mask_arr, dtype=np.float32)

    for x in [30, 60, 90]:
        map_dict[f"iauc{x}"] = np.zeros_like(tumor_mask_arr, dtype=np.float32)
        map_dict[f"iauc{x}_bn"] = np.zeros_like(tumor_mask_arr, dtype=np.float32)

    d_indeces, w_indeces, h_indeces = np.nonzero(tumor_mask_arr)

    for d,w,h in zip(d_indeces, w_indeces, h_indeces):

        voxel_T1w_t = img_arr[:, d, w, h]

        # Computing the BAT
        # bat_index = bat_fn.run(t_arr, voxel_T1w_t, search_interval(voxel_T1w_t)).bat_index # not necessary, better fit if passed the bat index on the mean signal

        # delay corrected extended Tofts
        t0_0 = t_arr[bat_index]
        vp_0 = 0.0025
        ktrans_0 = 0.058/60 #seconds-1
        ve_0 = 0.2 #fractional volume

        p_init = [t0_0, vp_0, ktrans_0, ve_0]
        p_lower = (-300.0, 1e-4, 60e-12/60, 0.01)
        p_upper = (300.0, 1.0, 3/60, 1.0)

        # Converting T1w to Concentration
        voxel_C_t = pkutils.T1w_to_Con(voxel_T1w_t, T1_0, TR, FA_deg, r1, 1)

        # PK Modelling and param estimation
        p_opt, p_cov = curve_fit(pkutils.delayCorrectedExtendedTofts(aif_fn), t_arr, voxel_C_t, p0 = p_init, bounds = [p_lower, p_upper], maxfev=100000)
        fitted_voxel_C_t = pkutils.delayCorrectedExtendedTofts(aif_fn)(t_arr, *p_opt)

        r2 = r2_score(voxel_C_t, fitted_voxel_C_t)

        t0, vp, ktrans, ve = p_opt #ktrans in sec -1
        kep = ktrans/ve #kep in sec -1

        map_dict["r2"][d, w, h] = r2
        map_dict["vp"][d, w, h] = vp
        map_dict["ktrans"][d, w, h] = ktrans * 60
        map_dict["ve"][d, w, h] = ve
        map_dict["kep"][d, w, h] = kep * 60
        map_dict["t0"][d, w, h] = t0

        for x in [30, 60, 90]:

            map_dict[f"iauc{x}"][d, w, h] = pkutils.get_iAUCx_with_functionalAIF(pkutils.ExtendedTofts, aif_fn, p_opt[1:], t_arr, t0, x=x)
            map_dict[f"iauc{x}_bn"][d, w, h] = pkutils.get_iAUCx_bn_with_functionalAIF(pkutils.ExtendedTofts, aif_fn, p_opt[1:], t_arr, t0, x=x)


    for key, map_arr in map_dict.items():

        map_img = sitk.GetImageFromArray(map_arr)
        map_img = align_header(sitk_tumor_mask, map_img)

        sitk.WriteImage(map_img, os.path.join(out_dir, f"{key}.nii.gz"))

    pbar.update()


In [ ]:
pbar = tqdm(sample_ids, position=0, desc="processing patients")
with ThreadPoolExecutor(NUM_WORKERS) as e:e.map(generate_pk_map, sample_ids)

processing patients:   8%|███▏                                    | 3/38 [09:02<1:36:57, 166.21s/it]

In [ ]:
# pbar = tqdm(sample_ids, position=0, desc="processing patients")
# for sample_id in sample_ids:
#     generate_pk_map(sample_id)

##### Voxel-wise Feature Extraction - Median, Std, 90th Percentile

In [ ]:
MAP_DIR = "pk_maps"

In [ ]:
class Percentile(object):
    
    def __init__(self, p):
        
        self.p = p
        
    def __call__(self, arr):
        
        return np.percentile(arr, self.p)

In [ ]:
feat_methods = {"median":np.median, "90percentile":Percentile(90), "std":np.std}

In [ ]:
voxel_pk_df = {"sample_id":[], "pk_map":[],  "median":[], "90percentile":[], "std":[], "pdl1>=1%":[], "pdl1>=50%":[]}

for sample_id in tqdm(sample_ids, position=0, desc="extracting features"):
    
    row_dict = stat_df[stat_df.ref_id==sample_id].iloc[0].to_dict()
    tps_50 = int(row_dict["pdl1>=50%"])
    tps_1 = int(row_dict["pdl1>=1%"])
    
    mask_img = sitk.ReadImage(os.path.join(DCE_DIR, "masks", sample_id, "mask.nii.gz"))
    mask_arr = sitk.GetArrayFromImage(mask_img)
    
    r2_img = sitk.ReadImage(os.path.join(MAP_DIR, sample_id, "r2.nii.gz"))
    r2_arr = sitk.GetArrayFromImage(r2_img)

    for pk_map in ["iauc30", "iauc60", "iauc90", "iauc30_bn", "iauc60_bn", "iauc90_bn", "t0", "kep", "ktrans", "ve", "vp", "r2"]:

        voxel_pk_df["sample_id"].append(sample_id)
        voxel_pk_df["pk_map"].append(pk_map)

        map_img = sitk.ReadImage(os.path.join(MAP_DIR, sample_id, f"{pk_map}.nii.gz"))
        map_arr = sitk.GetArrayFromImage(map_img)

        voxel_arr = map_arr[(mask_arr==1)&(r2_arr>0)] #r2_arr>0 is to exclude necrotic voxels
        for feat,feat_fn in feat_methods.items():
            voxel_pk_df[feat].append(feat_fn(voxel_arr))
            
        voxel_pk_df["pdl1>=1%"].append(tps_1)
        voxel_pk_df["pdl1>=50%"].append(tps_50)

voxel_pk_df = pd.DataFrame(voxel_pk_df)

##### Mann Whitney U-test (PDL1>=1%)

In [ ]:
label_column = "pdl1>=1%"

pvalue_df = {"feat":[], "p_value":[], "roc_auc":[]}

for pk_map in ["iauc30", "iauc60", "iauc90", "iauc30_bn", "iauc60_bn", "iauc90_bn", "t0", "kep", "ktrans", "ve", "vp", "r2"]:
    
    _pk_df = voxel_pk_df[voxel_pk_df.pk_map==pk_map] 
    
    for feat in ["median", "90percentile", "std"]:
    
        x1 = _pk_df[(_pk_df[label_column]==0)][feat].to_list()
        x2 = _pk_df[(_pk_df[label_column]==1)][feat].to_list()

        u, p_value = stats.mannwhitneyu(x1, x2)

        roc_auc = u/(len(x1)*len(x2))
        
        pvalue_df["feat"].append(pk_map+"_"+feat)
        pvalue_df["p_value"].append(p_value)
        pvalue_df["roc_auc"].append(roc_auc)

display(pd.DataFrame(pvalue_df))

##### Mann Whitney U-test (PDL1>=50%)

In [ ]:
label_column = "pdl1>=50%"

pvalue_df = {"feat":[], "p_value":[]}

for pk_map in ["iauc30", "iauc60", "iauc90", "iauc30_bn", "iauc60_bn", "iauc90_bn", "t0", "kep", "ktrans", "ve", "vp", "r2"]:
    
    _pk_df = voxel_pk_df[voxel_pk_df.pk_map==pk_map] 
    
    for feat in ["median", "90percentile", "std"]:
    
        x1 = _pk_df[(_pk_df[label_column]==0)][feat].to_list()
        x2 = _pk_df[(_pk_df[label_column]==1)][feat].to_list()

        u, p_value = stats.mannwhitneyu(x1, x2)

        roc_auc = u/(len(x1)*len(x2))
        
        pvalue_df["feat"].append(pk_map+"_"+feat)
        pvalue_df["p_value"].append(p_value)
  

pvalue_df = pd.DataFrame(pvalue_df)
_, pvalue_df["corr_p_value"], _, _ = multipletests(pvalue_df.p_value, method="fdr_bh")
display(pd.DataFrame(pvalue_df))